# Project Phase II

### Research Question(s)

A person's life expentancy is a given country can be predicted by different metrics of their country, including violent crime rates, health care, diet, economic development, genetics, and happiness level.

# Data Collection and Cleaning

Have an initial draft of your data cleaning
appendix. Document every step that takes your raw data file(s) and turns it
into the analysis-ready data set that you would submit with your final
project. Include text cells describing your data collection (downloading,
scraping, surveys, etc), and text cells describing any additional data
curation/cleaning (merging data frames, filtering, transformations of
variables, etc). Include code cells for data curation/cleaning, but not
collection. Note: You should be saving data in intermediate files at several
points through this process so that you are not starting from scratch every
time you change something. This also makes sure you have
documentation for everything that you have done for your reference, but
we do not necessarily need to see all of it in code form.

- Ryan: life expectancies, violent crime
- John: health care, diet, economic develpment
- Chase: genetics, happiness level

1. Find all data
- Each person does their assigned data set
2. Clean data sets
- Each person does their assigned data set
3. Group data sets (if needed)
- Each person does their assigned data set
4. Join data sets
- Chase

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import requests
from bs4 import BeautifulSoup

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Life Expectancy
#### https://data.worldbank.org/indicator/SP.DYN.CDRT.IN
I found and used this dataset after discovering that the original data we wanted to use from the UN only allowed 100k rows to be downloaded. I downloaded this data and filtered down to the two columns we were looking for, country name, and the most recent year of data. I then renamed these columns to be simplified to "Country" and "Life Expectancy".

In [3]:
df_life_expectancies = pd.read_csv("data/life-expectancies.csv")
df_life_expectancies = df_life_expectancies[["Country Name","2020"]].rename(columns={"Country Name" : "Country","2020" : "Life Expectancy"})
df_life_expectancies.head(5)

,Country,Life Expectancy
0,Aruba,76.434000
1,Africa Eastern and Southern,64.325702
2,Afghanistan,65.173000
3,Africa Western and Central,58.445953
4,Angola,61.487000


## Intentional Homicide Rate
#### https://worldpopulationreview.com/country-rankings/murder-rate-by-country
I have found and used the violent crime dataset from the world population review. I then downloaded it and read it into a pandas data frame. I think narrowed it down to our columns of interest and renamed them.

In [4]:
df_crime = pd.read_csv("data/violent-crime-rates.csv")
df_crime = df_crime[["country","homRate"]].rename(columns={"country":"Country","homRate":"Homicide Rate"})
df_crime.head()

,Country,Homicide Rate
0,El Salvador,52.02
1,United States Virgin Islands,49.28
2,Jamaica,43.85
3,Lesotho,43.56
4,Honduras,38.93


## Economic Development
#### https://hdr.undp.org/data-center/human-development-index#/indicies/HDI
From the site: "The Human Development Index (HDI) is a summary measure of average achievement in key dimensions of human development: a long and healthy life, being knowledgeable and have a decent standard of living. The HDI is the geometric mean of normalized indices for each of the three dimensions."

Included later on is GNI per capita, the average income of an individual for the country, and is Purchasing Power Parity(PPP)-adjusted, and life expectancy.

In [5]:
evens_only = np.arange(2,32,2)
add_unnamed = lambda x : "Unnamed: "+ str(x)
evens_only = list(map(add_unnamed, evens_only))
evens_only.remove("Unnamed: 24")
evens_only.append("SDG10.1")
hdi_df = pd.read_excel("data/HDR21-22_Statistical_Annex_I-HDI_Table.xlsx", 
#                       )
                       usecols=["Table 3. Inequality-adjusted Human Development Index "]+evens_only)

#clean up column names
hdi_df = hdi_df[1:]
headers = hdi_df.iloc[0]
header_types = hdi_df.iloc[1]
for i, header in enumerate(headers):
    header = str(header) + ", " if str(header) != "nan" else ""
    header_types[i] = str(header_types[i]) if header_types[i] != "nan" else ""
    headers[i] = f'{header}{header_types[i]}'
    
hdi_df = hdi_df[3:].rename(columns=headers)
hdi_df = hdi_df.rename(columns={hdi_df.columns[0]:"Country"})
hdi_df = hdi_df[hdi_df["Country"].notnull()]

#partition with respect to levels of human development 
levels = hdi_df[hdi_df["Country"].str.contains("human development", na=False)]
# print(levels['Country'].index)
#4: v high, 71: high, 121: med, 166: low
#206-209: aggregates; 210-226: regions avgs (but no country:region mapping, so it's kinda useless) 
hdi_df["Level of human development"] = "Very high"
hdi_df.loc[71:121,"Level of human development"] = "High"
hdi_df.loc[121:166,"Level of human development"] = "Medium"
hdi_df.loc[166:204,"Level of human development"] = "Low"
hdi_df.loc[200:,"Level of human development"] = "n/a"

hdi_df = hdi_df[:200] #note: cutting off region data
hdi_df.iloc[-10:]
hdi_df = hdi_df.dropna(axis=0) #drop the "human development" header rows (rows with NaN's)
hdi_df[:-50]

#replace periods with -99.9 to allow for sorting
hdi_df = hdi_df.replace(to_replace=r'^\.+', value=-99.9, regex=True) 

# hdi_df = hdi_df.sort_values(by="Overall loss (%)", ascending=False)
# hdi_df[hdi_df["Country"] == "United States"]
hdi_df = hdi_df.reset_index(drop=True)
hdi_df.head()
#convert all strings to 0

# hdi_df[207:]
# hdi_df.shape
# hdi_df[239:]
# hdi_df[206:] #regions are from 210 to 226 

,Country,"Human Development Index (HDI), Value","Inequality-adjusted HDI (IHDI), Value",Overall loss (%),Difference from HDI rank,"Coefficient of human inequality, nan","Inequality in life expectancy, (%)","Inequality-adjusted life expectancy index, Value","Inequality in education, (%)","Inequality-adjusted education index, Value","Inequality in income, (%)","Inequality-adjusted income index, Value","Income shares held by (%), Poorest 40 percent",Richest 10 percent,Richest 1 percent,Gini coefficient,Level of human development
0,Switzerland,0.962,0.894,7.068607,-3,6.899906,3.103429,0.953868,2.01457,0.90179,15.58172,0.829672,19.9,25.8,11.46,33.1,Very high
1,Norway,0.961,0.908,5.515088,0,5.435152,2.541615,0.948104,2.3324,0.911682,11.43144,0.865839,22.9,22.4,8.88,27.7,Very high
2,Iceland,0.959,0.915,4.588113,2,4.556437,1.954472,0.945433,2.20077,0.937802,9.51407,0.864396,23.9,22.1,8.78,26.1,Very high
3,"Hong Kong, China (SAR)",0.952,0.828,13.02521,-19,12.429978,2.055595,0.979444,9.671619,0.801655,25.56272,0.724064,-99.9,-99.9,17.85,-99.9,Very high
4,Australia,0.951,0.876,7.886435,-6,7.640183,2.73798,0.965535,3.05657,0.895978,17.126,0.77606,19.5,26.6,11.28,34.3,Very high


#### Life Expectancies and other indicators
- UN dataset again, this time having columns like life expectancy and GNI per capita

In [6]:
#life expectancies and whatnot
evens_only = np.arange(2,16,2)
add_unnamed = lambda x : "Unnamed: "+ str(x)
evens_only = list(map(add_unnamed, evens_only))
evens_only += ["Table 1. Human Development Index and its components "]
life_df = pd.read_excel("data/HDR21-22_Statistical_Annex_HDI_Table.xlsx", usecols=evens_only)
life_df[:10]
col_names = life_df.iloc[3]
life_df = life_df[1:].rename(columns=col_names)
life_df = life_df.dropna(axis=0) #drop NaN rows -- the "high development countries" header thing gets dropped too
life_df = life_df.rename(columns={life_df.columns[0]: "Country", "Human Development Index (HDI) ": "Human Development Index (HDI)"})
# life_df.sort_values(by=['Country']).head()
life_df = life_df.reset_index(drop=True)
life_df = life_df[:195]
life_df = life_df.replace(to_replace=r'^\.+', value=-99.9, regex=True)
life_df.head()

,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
0,Switzerland,0.962,83.9872,16.500299,13.85966,66933.00454,5,3
1,Norway,0.961,83.2339,18.1852,13.00363,64660.10622,6,1
2,Iceland,0.959,82.6782,19.163059,13.76717,55782.04981,11,2
3,"Hong Kong, China (SAR)",0.952,85.4734,17.27817,12.22621,62606.8454,6,4
4,Australia,0.951,84.5265,21.05459,12.72682,49238.43335,18,5


In [7]:
#Join the two datasets on country name 
print(f"should be same # rows b/c they're from the same site: \nhdi_df: {hdi_df.shape}, life_df: {life_df.shape}")
full_hdi_df = hdi_df.join(life_df.set_index('Country'), on="Country")
print(f'full_hdi_df:{full_hdi_df.shape}')
full_hdi_df = full_hdi_df.drop(columns="Human Development Index (HDI)") #drop extra HDI col
full_hdi_df.columns

#remove "Value" and commas from column names
cols_edit = list(map(lambda x: x.replace(", Value", ""), full_hdi_df.columns))
cols_edit = list(map(lambda x: x.replace(",", ""), cols_edit))
col_dict = {}
for i, col in enumerate(full_hdi_df):
    col_dict[col] = cols_edit[i]
full_hdi_df = full_hdi_df.rename(columns=col_dict)
full_hdi_df = full_hdi_df.sort_values(by="Country")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(full_hdi_df)

# full_hdi_df.columns

should be same # rows b/c they're from the same site: 
hdi_df: (195, 17), life_df: (195, 8)
full_hdi_df:(195, 24)


,Country,Human Development Index (HDI),Inequality-adjusted HDI (IHDI),Overall loss (%),Difference from HDI rank,Coefficient of human inequality nan,Inequality in life expectancy (%),Inequality-adjusted life expectancy index,Inequality in education (%),Inequality-adjusted education index,Inequality in income (%),Inequality-adjusted income index,Income shares held by (%) Poorest 40 percent,Richest 10 percent,Richest 1 percent,Gini coefficient,Level of human development,Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
179,Afghanistan,0.478,-99.9,-99.9,-99.9,-99.9,26.165367,0.476885,45.36517,0.210131,-99.9,-99.9,-99.9,-99.9,15.25,-99.9,Low,61.9824,10.263844,2.98507,1824.190915,-2,177
66,Albania,0.796,0.71,10.80402,5,10.767685,6.790635,0.809668,12.33344,0.68165,13.17898,0.64931,21,23.8,8.91,30.8,High,76.4626,14.448,11.286455,14131.11039,17,68
90,Algeria,0.745,0.598,19.731544,-7,19.072875,12.430804,0.759517,33.28262,0.450528,11.5052,0.625894,23.1,22.9,9.91,27.6,High,76.3767,14.626896,8.069284,10800.22546,13,96
39,Andorra,0.858,-99.9,-99.9,-99.9,-99.9,5.29791,0.879541,10.008154,0.649101,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,Very high,80.3684,13.300239,10.55512,51166.62661,-19,45
147,Angola,0.586,0.407,30.546075,-2,30.412168,28.187733,0.460078,34.17144,0.341449,28.87733,0.429854,11.5,39.6,25.98,51.3,Medium,61.6434,12.1721,5.417391,5465.617791,-14,149
70,Antigua and Barbuda,0.788,-99.9,-99.9,-99.9,-99.9,4.746306,0.857236,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,High,78.4968,14.184626,9.293741,16792.36595,2,71
46,Argentina,0.842,0.72,14.489311,-6,13.837301,7.624846,0.787177,5.78729,0.81786,28.099767,0.580359,14.3,30.3,21.7,42.3,Very high,75.3899,17.87487,11.147269,20925.26814,17,47
84,Armenia,0.759,0.688,9.354414,13,9.136398,7.035874,0.744329,2.93495,0.720253,17.43837,0.608557,24.5,21.5,15.39,25.2,High,72.0431,13.11676,11.3303,13157.9939,4,87
4,Australia,0.951,0.876,7.886435,-6,7.640183,2.73798,0.965535,3.05657,0.895978,17.126,0.77606,19.5,26.6,11.28,34.3,Very high,84.5265,21.05459,12.72682,49238.43335,18,5
24,Austria,0.916,0.851,7.09607,9,6.90408,3.256919,0.916525,2.52241,0.8317,14.93291,0.807548,21.2,23.2,10.09,30.2,Very high,81.5797,16.007959,12.25669,53618.67138,-8,23


In [8]:
#correct the country names w.r.t. Wikipedia
changes_dict = {
    "Bolivia (Plurinational State of)": "Bolivia",
    "Congo (Democratic Republic of the)" : "Democratic Republic of the Congo",
    "Congo" : "Republic of the Congo",
    "Hong Kong, China (SAR)" : "Hong Kong",
    "Iran (Islamic Republic of)" : "Iran",
    "Korea (Democratic People's Rep. of)" : "North Korea",
    "Korea (Republic of)" : "South Korea",
    "Lao People's Democratic Republic" : "Laos",
    "Moldova (Republic of)" : "Moldova",
    "Russian Federation" : "Russia",
    "Syrian Arab Republic" : "Syria",
    "Tanzania (United Republic of)" : "Tanzania",
    "Venezuela (Bolivarian Republic of)" : "Venezuela",
    "Viet Nam" : "Vietnam",
    "United States" : "United States of America",
    "Türkiye": "Turkey",
    "Gambia": "The Gambia",
    "Eswatini (Kingdom of)": "Eswatini",
    "Czechia" : "Czech Republic",
    "Brunei Darussalam": "Brunei"
}
for old, new in changes_dict.items():
    full_hdi_df.loc[full_hdi_df['Country'] == old, "Country"] = new

full_hdi_df = full_hdi_df.sort_values(by="Country")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(full_hdi_df)

,Country,Human Development Index (HDI),Inequality-adjusted HDI (IHDI),Overall loss (%),Difference from HDI rank,Coefficient of human inequality nan,Inequality in life expectancy (%),Inequality-adjusted life expectancy index,Inequality in education (%),Inequality-adjusted education index,Inequality in income (%),Inequality-adjusted income index,Income shares held by (%) Poorest 40 percent,Richest 10 percent,Richest 1 percent,Gini coefficient,Level of human development,Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
179,Afghanistan,0.478,-99.9,-99.9,-99.9,-99.9,26.165367,0.476885,45.36517,0.210131,-99.9,-99.9,-99.9,-99.9,15.25,-99.9,Low,61.9824,10.263844,2.98507,1824.190915,-2,177
66,Albania,0.796,0.71,10.80402,5,10.767685,6.790635,0.809668,12.33344,0.68165,13.17898,0.64931,21,23.8,8.91,30.8,High,76.4626,14.448,11.286455,14131.11039,17,68
90,Algeria,0.745,0.598,19.731544,-7,19.072875,12.430804,0.759517,33.28262,0.450528,11.5052,0.625894,23.1,22.9,9.91,27.6,High,76.3767,14.626896,8.069284,10800.22546,13,96
39,Andorra,0.858,-99.9,-99.9,-99.9,-99.9,5.29791,0.879541,10.008154,0.649101,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,Very high,80.3684,13.300239,10.55512,51166.62661,-19,45
147,Angola,0.586,0.407,30.546075,-2,30.412168,28.187733,0.460078,34.17144,0.341449,28.87733,0.429854,11.5,39.6,25.98,51.3,Medium,61.6434,12.1721,5.417391,5465.617791,-14,149
70,Antigua and Barbuda,0.788,-99.9,-99.9,-99.9,-99.9,4.746306,0.857236,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,-99.9,High,78.4968,14.184626,9.293741,16792.36595,2,71
46,Argentina,0.842,0.72,14.489311,-6,13.837301,7.624846,0.787177,5.78729,0.81786,28.099767,0.580359,14.3,30.3,21.7,42.3,Very high,75.3899,17.87487,11.147269,20925.26814,17,47
84,Armenia,0.759,0.688,9.354414,13,9.136398,7.035874,0.744329,2.93495,0.720253,17.43837,0.608557,24.5,21.5,15.39,25.2,High,72.0431,13.11676,11.3303,13157.9939,4,87
4,Australia,0.951,0.876,7.886435,-6,7.640183,2.73798,0.965535,3.05657,0.895978,17.126,0.77606,19.5,26.6,11.28,34.3,Very high,84.5265,21.05459,12.72682,49238.43335,18,5
24,Austria,0.916,0.851,7.09607,9,6.90408,3.256919,0.916525,2.52241,0.8317,14.93291,0.807548,21.2,23.2,10.09,30.2,Very high,81.5797,16.007959,12.25669,53618.67138,-8,23


## Diet 
- meat consumption: https://en.wikipedia.org/wiki/List_of_countries_by_meat_consumption
    - try to get most recent, non-2019 (pandemic year) data
    - set to -99.9 otherwise
- caloric intake: https://en.wikipedia.org/wiki/List_of_countries_by_food_energy_intake
    - set to -99.9 if the country has no data
    
- `kg meat/person`: yearly consumption of meat per person in kg
- `cal`: daily caloric intake

In [10]:
#meat consumption by country (2017 or earlier)
meat_url = 'https://en.wikipedia.org/wiki/List_of_countries_by_meat_consumption'
meat_rq = requests.get(meat_url)
if meat_rq.status_code != 200:
  print("something went wrong:", meat_rq.status_code, meat_rq.reason)

with open("data/meat_rq.html", "w") as writer: 
  writer.write(meat_rq.text)

with open("data/meat_rq.html", "r") as reader:
  meat_source = reader.read()

meat_soup = BeautifulSoup(meat_source, "html.parser")
table = meat_soup.find("table", {"class": "wikitable"})
meat_dict = {"Country": [], "kg meat/person":[]}
for row in table.findAll("tr")[1:]: 
    meat_dict['Country'].append(row.find("a").text)
    
    #try to find most recent year with data, and set to -99.9 otherwise
    vals = row.findAll("td")
    val_final = -99.9
    for val in vals[-2::-1]: #iterate backwards for most recent; skip 2019 b/c of pandemic
        try:
            val_final = float(val.text)
        except:
            continue 
        break
    meat_dict['kg meat/person'].append(val_final)
    
meat_df = pd.DataFrame.from_dict(data=meat_dict)
# print(meat_df.shape)

#fix some names
meat_df.loc[meat_df["Country"] == "Cape Verde", "Country"] = "Cabo Verde"
meat_df.loc[meat_df["Country"] == "American Samoa", "Country"] = "Samoa"
meat_df.loc[meat_df["Country"] == "Swaziland", "Country"] = "Eswatini"

#join with HDI and drop the NaN's
full_df = full_hdi_df.join(meat_df.set_index('Country'), on="Country")
full_df = full_df.fillna(-99.9)

#print countries that were left out 
un_countries = list(full_df['Country'])
not_in = ""
for country in meat_df["Country"]:
    if country not in un_countries:
        not_in += f"{country}, "
    else: pass
print(f"Countries in meat_df but not UN's dataset: {not_in}")

#display to check
# full_df = full_df.sort_values(by="Country")
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(full_df)

Countries in meat_df but not UN's dataset: French Guiana, French Polynesia, Guadeloupe, Macao, Netherlands Antilles, New Caledonia, Reunion, FR Yugoslavia, Taiwan, 


In [11]:
#caloric intake by country (2018)
cal_url = 'https://en.wikipedia.org/wiki/List_of_countries_by_food_energy_intake'
cal_rq = requests.get(cal_url)
if cal_rq.status_code != 200:
  print("something went wrong:", cal_rq.status_code, cal_rq.reason)

with open("data/cal_rq.html", "w") as writer: 
  writer.write(cal_rq.text)

with open("data/cal_rq.html", "r") as reader:
  cal_source = reader.read()

cal_soup = BeautifulSoup(cal_source, "html.parser")
table = cal_soup.find("table", {"class": "wikitable"})
cal_dict = {"Country": [], "cal":[]}
for row in table.findAll("tr")[2:]: #skip the two headers and check each row
    cal_dict['Country'].append(row.find("a").text)
    
    #rstrip() to remove the newline characters, and remove commas
    cal_dict['cal'].append(float(row.findAll("td")[2].text.rstrip().replace(",","")))
    
cal_df = pd.DataFrame.from_dict(data=cal_dict)

#fix some names
cal_df.loc[cal_df["Country"] == "United States", "Country"] = "United States of America"
cal_df.loc[cal_df["Country"] == "Gambia", "Country"] = "The Gambia"
cal_df.loc[cal_df["Country"] == "São Tomé and Príncipe", "Country"] = "Sao Tome and Principe"
cal_df.loc[cal_df["Country"] == "Ivory Coast", "Country"] = "Côte d'Ivoire"
cal_df.loc[cal_df["Country"] == "Cape Verde", "Country"] = "Cabo Verde"

#join with everything else 
full_df1 = full_df.join(cal_df.set_index('Country'), on="Country")
full_df1 = full_df1.fillna(-99.9)

#print countries that were left out 
un_countries = list(full_df1['Country'])
not_in = ""
for country in cal_df["Country"]:
    if country not in un_countries:
        not_in += f"{country}, "
    else: pass
print(f"Countries in cal_df but not UN's dataset: {not_in}")
    

#display to check
# full_df1 = full_df1.sort_values(by="Country")
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(full_df1)

Countries in cal_df but not UN's dataset: Taiwan, New Caledonia, 


## Healthcare
- has/does not have universal health care (UHC): https://en.wikipedia.org/wiki/List_of_countries_with_universal_health_care
- % of spending on health w.r.t. GDP:
https://data.worldbank.org/indicator/SH.XPD.CHEX.GD.ZS?most_recent_value_desc=true

- `has_uhc`: 1 → country has universal healthcare, 0 → doesn't have UHC

In [13]:
#UHC or not
uhc_url = 'https://en.wikipedia.org/wiki/List_of_countries_with_universal_health_care'
uhc_rq = requests.get(uhc_url)
if uhc_rq.status_code != 200:
  print("something went wrong:", uhc_rq.status_code, uhc_rq.reason)

with open("data/uhc_rq.html", "w") as writer: 
  writer.write(uhc_rq.text)

with open("data/uhc_rq.html", "r") as reader:
  uhc_source = reader.read()

uhc_soup = BeautifulSoup(uhc_source, "html.parser")
spans = uhc_soup.findAll("span", {"class": "toctext"})
uhc_dict = {"Country": [], "has_uhc": []}
for span in spans: 
    uhc_dict['Country'].append(span.text)
    uhc_dict['has_uhc'].append(1)
    
uhc_df = pd.DataFrame.from_dict(data=uhc_dict)
# uhc_df.shape

#fix some names
uhc_df.loc[uhc_df["Country"] == "South Korea (ROK)", "Country"] = "South Korea"
uhc_df.loc[uhc_df["Country"] == "North Korea (DPRK)", "Country"] = "North Korea"
#England, Scotland, Wales, Northern Ireland → United Kingdom -- only need one to change though
uhc_df.loc[uhc_df["Country"] == "England", "Country"] = "United Kingdom"

#join with everything; fill with 0 if the country doesn't have UHC
full_df2 = full_df1.join(uhc_df.set_index('Country'), on="Country")
full_df2 = full_df2.fillna(0)

#print countries that were left out 
un_countries = list(full_df1['Country'])
not_in = ""
for country in uhc_df["Country"]:
    if country in ["Northern Ireland", "Scotland", "Wales"]:
        continue
    if country not in un_countries:
        not_in += f"{country}, "
    else: 
        continue
print(f"Rows in uhc_df but not UN's dataset: {not_in}")
    
#display to check
full_df2 = full_df2.sort_values(by="has_uhc")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(full_df2)
    
full_df2.to_csv("data/indicators.csv")

Rows in uhc_df but not UN's dataset: Africa, Asia, Macau, Taiwan, Europe, Crown dependencies, Isle of Man, Guernsey/Jersey, North and Central America, The Bahamas, South America, Oceania, See also, References, 


,Country,Human Development Index (HDI),Inequality-adjusted HDI (IHDI),Overall loss (%),Difference from HDI rank,Coefficient of human inequality nan,Inequality in life expectancy (%),Inequality-adjusted life expectancy index,Inequality in education (%),Inequality-adjusted education index,Inequality in income (%),Inequality-adjusted income index,Income shares held by (%) Poorest 40 percent,Richest 10 percent,Richest 1 percent,Gini coefficient,Level of human development,Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank,kg meat/person,cal,has_uhc
179,Afghanistan,0.478,-99.900,-99.900000,-99.9,-99.900000,26.165367,0.476885,45.365170,0.210131,-99.900000,-99.900000,-99.9,-99.9,15.25,-99.9,Low,61.9824,10.263844,2.985070,1824.190915,-2.0,177.0,-99.90,2040.0,0.0
80,Moldova,0.767,0.711,7.301173,16.0,7.297466,8.885082,0.684706,2.570094,0.774538,10.437222,0.676758,24.1,22.0,9.81,26.0,High,68.8459,14.433000,11.821590,14875.331890,-1.0,81.0,38.12,2383.0,0.0
192,Monaco,-99.900,-99.900,-99.900000,-99.9,-99.900000,3.428433,0.965716,-99.900000,-99.900000,-99.900000,-99.900000,-99.9,-99.9,-99.90,-99.9,n/a,85.9463,-99.900000,-99.900000,-99.900000,-99.9,-99.9,-99.90,-99.9,0.0
95,Mongolia,0.739,0.644,12.855210,10.0,12.725367,9.346192,0.710941,11.883630,0.643465,16.946280,0.584924,20.2,25.7,16.54,32.7,High,70.9754,14.980350,9.423700,10588.225320,9.0,90.0,88.38,2579.0,0.0
48,Montenegro,0.832,0.756,9.134615,2.0,9.010793,2.293319,0.846931,7.815010,0.760357,16.924050,0.670047,16.8,26.0,9.69,36.8,Very high,76.3426,15.081610,12.176483,20838.801290,16.0,52.0,73.01,3500.0,0.0
184,Mozambique,0.446,0.300,32.735426,0.0,32.408423,28.299297,0.433786,40.531431,0.232197,28.394540,0.268604,11.8,45.5,31.11,54.0,Low,59.3247,10.219152,3.197642,1198.073924,2.0,184.0,9.39,2103.0,0.0
148,Myanmar,0.585,-99.900,-99.900000,-99.9,-99.900000,21.222845,0.553520,26.852720,0.377141,-99.900000,-99.900000,21.9,25.5,17.09,30.7,Medium,65.6716,10.909138,6.376800,3850.524234,5.0,145.0,58.25,2673.0,0.0
138,Namibia,0.615,0.402,34.634146,-10.0,32.839455,19.917114,0.483812,25.015670,0.428194,53.585580,0.312574,8.6,47.3,21.57,59.1,Medium,59.2690,11.927207,7.192013,8633.504452,-23.0,134.0,30.53,2469.0,0.0
193,Nauru,-99.900,-99.900,-99.900000,-99.9,-99.900000,13.459402,0.580714,-99.900000,-99.900000,-99.900000,-99.900000,19.4,27.3,-99.90,34.8,n/a,63.6170,11.690420,-99.900000,17729.740840,-99.9,-99.9,-99.90,-99.9,0.0
142,Nepal,0.602,0.449,25.415282,0.0,24.319129,15.541866,0.629531,41.099200,0.311407,16.316320,0.462370,20.4,26.4,13.89,32.8,Medium,68.4495,12.886950,5.121793,3877.315444,10.0,144.0,13.55,2769.0,0.0


In [642]:
# % of healthcare spending wrt GDP
spend_df = pd.read_csv("data/API_SH.XPD.CHEX.GD.ZS_DS2_en_csv_v2_4666562.csv",
                      usecols=[
                          "Country Name", 
                          "2016",
                          "2017",
                          "2018",
                          "2019"
                      ]) 
spend_df = spend_df.rename(columns={"Country Name": "Country"})

#set the "% gdp on healthcare" column to most recent value (???????)
spend_df = spend_df.fillna(-1)
# spend_df["% gdp on healthcare"] = spend_df[""]

#fix some names
uhc_df.loc[uhc_df["Country"] == "South Korea (ROK)", "Country"] = "South Korea"
#Cote d'Ivoire, Congo, Dem. Rep., Congo, Rep.,American Samoa, Brunei Darussalam, Korea, Rep.
#Hong Kong SAR, China // Korea, Dem. People's Rep., //Iran, Islamic Rep. // Gambia, The

"""
have some dict with {new_name: standard_name} pairs? 
e.g.: master_dict = {
    "United States": "United States of America",
    "USA": "United States of America",
    ... 
}

then, use df.map(remap_with_master_dict, df) 
to make another dict with standardized names → super easy joins
"""


#join with everything
full_df3 = full_df2.join(spend_df.set_index('Country'), on="Country")

# print countries that were left out 
un_countries = list(full_df1['Country'])
not_in = ""
for country in spend_df["Country"]:
    if country not in un_countries:
        not_in += f"{country}// "
    else: 
        continue
print(f"Rows in spend_df but not UN's dataset: {not_in}")



# spend_df[:20]

Rows in spend_df but not UN's dataset: Aruba// Africa Eastern and Southern// Africa Western and Central// Arab World// American Samoa// Bahamas, The// Bermuda// Brunei Darussalam// Central Europe and the Baltics// Channel Islands// Cote d'Ivoire// Congo, Dem. Rep.// Congo, Rep.// Caribbean small states// Curacao// Cayman Islands// Czechia// East Asia & Pacific (excluding high income)// Early-demographic dividend// East Asia & Pacific// Europe & Central Asia (excluding high income)// Europe & Central Asia// Egypt, Arab Rep.// Euro area// European Union// Fragile and conflict affected situations// Faroe Islands// Micronesia, Fed. Sts.// Gibraltar// Gambia, The// Greenland// Guam// High income// Hong Kong SAR, China// Heavily indebted poor countries (HIPC)// IBRD only// IDA & IBRD total// IDA total// IDA blend// IDA only// Isle of Man// Not classified// Iran, Islamic Rep.// Kyrgyz Republic// St. Kitts and Nevis// Korea, Rep.// Latin America & Caribbean (excluding high income)// Lao PDR// 

In [18]:
john_final = pd.read_csv("data/indicators.csv").sort_values(by="Human Development Index (HDI)", ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(john_final)
#kg meat/person: yearly consumption of meat per person in kg
#cal: daily caloric intake
#has_uhc: 1 → country has universal healthcare, 0 → doesn't have UHC

,Unnamed: 0,Country,Human Development Index (HDI),Inequality-adjusted HDI (IHDI),Overall loss (%),Difference from HDI rank,Coefficient of human inequality nan,Inequality in life expectancy (%),Inequality-adjusted life expectancy index,Inequality in education (%),Inequality-adjusted education index,Inequality in income (%),Inequality-adjusted income index,Income shares held by (%) Poorest 40 percent,Richest 10 percent,Richest 1 percent,Gini coefficient,Level of human development,Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank,kg meat/person,cal,has_uhc
146,0,Switzerland,0.962,0.894,7.068607,-3.0,6.899906,3.103429,0.953868,2.014570,0.901790,15.581720,0.829672,19.9,25.8,11.46,33.1,Very high,83.9872,16.500299,13.859660,66933.004540,5.0,3.0,67.54,3354.0,1.0
190,1,Norway,0.961,0.908,5.515088,0.0,5.435152,2.541615,0.948104,2.332400,0.911682,11.431440,0.865839,22.9,22.4,8.88,27.7,Very high,83.2339,18.185200,13.003630,64660.106220,6.0,1.0,67.47,3371.0,1.0
166,2,Iceland,0.959,0.915,4.588113,2.0,4.556437,1.954472,0.945433,2.200770,0.937802,9.514070,0.864396,23.9,22.1,8.78,26.1,Very high,82.6782,19.163059,13.767170,55782.049810,11.0,2.0,91.02,3654.0,1.0
165,3,Hong Kong,0.952,0.828,13.025210,-19.0,12.429978,2.055595,0.979444,9.671619,0.801655,25.562720,0.724064,-99.9,-99.9,17.85,-99.9,Very high,85.4734,17.278170,12.226210,62606.845400,6.0,4.0,137.08,3267.0,1.0
139,4,Australia,0.951,0.876,7.886435,-6.0,7.640183,2.737980,0.965535,3.056570,0.895978,17.126000,0.776060,19.5,26.6,11.28,34.3,Very high,84.5265,21.054590,12.726820,49238.433350,18.0,5.0,121.61,3391.0,1.0
193,5,Denmark,0.948,0.898,5.274262,3.0,5.186914,3.006301,0.915849,2.503760,0.908681,10.050680,0.869997,23.5,23.5,12.91,27.7,Very high,81.3753,18.714800,12.960490,60364.785950,6.0,5.0,70.83,3401.0,1.0
147,6,Sweden,0.947,0.885,6.546990,0.0,6.416555,2.610125,0.943682,3.867900,0.884727,12.771640,0.830188,21.9,22.7,10.54,29.3,Very high,82.9833,19.418530,12.609720,54489.374010,9.0,9.0,77.07,3184.0,1.0
171,7,Ireland,0.945,0.886,6.243386,2.0,6.161783,2.825730,0.926857,3.396660,0.855977,12.262960,0.877370,21.8,25.1,11.80,30.6,Very high,81.9976,18.945221,11.582223,76168.984430,-3.0,8.0,76.85,3885.0,1.0
160,8,Germany,0.942,0.883,6.263270,1.0,6.177130,3.408159,0.900980,2.663200,0.917110,12.460030,0.833262,20.8,25.1,12.77,31.7,Very high,80.6301,17.010139,14.090967,54534.216820,6.0,7.0,87.79,3554.0,1.0
175,9,Netherlands,0.941,0.878,6.695005,1.0,6.697945,3.334346,0.917391,4.866740,0.874643,11.892750,0.842143,22.3,23.9,6.92,29.2,Very high,81.6873,18.693165,12.581630,55979.411000,3.0,10.0,75.82,3297.0,1.0


### Data Description

Have an initial draft of your data description (link in syllabus) section.
Your data description should be about your analysis-ready data.

### Data Limitations

Identify any potential problems with your dataset.

### Exploratory Data Analysis

Perform an (initial) exploratory data analysis.

1. Collinearity test
2. Multivariate regression model
3. Plots
4. Summary stats

### Questions for Reviewers

List specific questions for your peer reviewers
and project mentor to answer in giving you feedback on this phase.